In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
from src import parse

In [ ]:
restaurants = parse.parse_from_restaurants_csv('../../examples/boston/20200405/restaurants.csv')

In [ ]:
restaurants.shape[1]

In [ ]:
restaurants

In [ ]:
hospitals = parse.parse_from_hospital_requests_csv('../../examples/boston/20200405/hospitals.csv')

In [ ]:
hospitals.shape[1]

In [ ]:
hospitals

In [ ]:
zone_groups = hospitals.groupby('zone')

In [ ]:
for z, g in zone_groups:
    print(z, g)

In [ ]:
schedule = hospitals.iloc[:, 5:]

In [ ]:
schedule

In [ ]:
results = parse.parse_from_results_csv('../../examples/boston/20200405/results.csv')

In [ ]:
results_schedule = results.iloc[:, 5:]
results_schedule_columns = results_schedule.columns
rest_usage = restaurants.copy()

In [ ]:
log = []
for i, row in results_schedule.iterrows():
    for j, val in enumerate(row):
        if not pd.isnull(val):
            log.append([hospitals.iloc[i, 1], results_schedule_columns[j], val, schedule.iloc[i, j]])


In [ ]:
meal_log = pd.DataFrame(log, columns=['hospital', 'mea', 'restaurant', 'quantity'])
meal_log

In [ ]:
meal_log['quantity'].sum()

In [ ]:
meal_log.groupby('restaurant').sum().sort_values(by='quantity', ascending=False)

In [ ]:
meal_log.groupby('hospital').sum().sort_values(by='quantity', ascending=False)